In [1]:
import numpy as np
import random
import pandas as pd
from scipy.stats import skewnorm

import networkx as nx

import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
from ndlib.utils import multi_runs

Algorithms used in aggregation simulation on synthetic networks

In [2]:
def join_nodes(G, node_0, node_1): # aggregating two nodes in graph G

    edges_0 = G.edges(node_0, data=True)
    edges_1 = G.edges(node_1, data=True) # edges for the two nodes
    
    new_edges = []

    for i, (s, t, data) in enumerate(edges_1): # taking edges from node 1 and moving them to node 0        
        if (s!=t):
            new_edges.append((node_0, t, data))
    
    G.remove_node(node_1)
    G.add_edges_from(new_edges)
    
    return G

def shared_neighbours(G, node_0, node_1, min_share = True): # calculating proportion of shared neighbours
    neighbours_0 = [n for n in G.neighbors(node_0)]
    neighbours_0 = [x for x in neighbours_0 if x != node_1]
    neighbours_1 = [n for n in G.neighbors(node_1)]
    neighbours_1 = [x for x in neighbours_1 if x != node_0]
    shared_nodes = [x for x in neighbours_0 if x in neighbours_1]
    if (len(neighbours_0)*len(neighbours_1)!=0):
        if min_share: # whether to use smaller or larger proportion shared neighbours
            return min(len(shared_nodes)/len(neighbours_0), len(shared_nodes)/len(neighbours_1))
        else:
            return max(len(shared_nodes)/len(neighbours_0), len(shared_nodes)/len(neighbours_1))
    else:
        return 0

Node aggregation, SIR model

In [3]:
# proportion of population infected for different aggregation
rand_agg = []
non_rand_agg = []
full_res = []

nodes_aggregated = []
graph_num = []

#average degrees
non_rand_ave_deg = []
rand_ave_deg = []

# peak infected numbers
peak_inf_full = []
peak_inf_non_rand = []
peak_inf_rand = []

# times to stability
stability_full = []
stability_non_rand = []
stability_rand = []

# times to peak infection
peak_time_full = []
peak_time_non_rand = []
peak_time_rand = []

config = mc.Configuration()
config.add_model_parameter('beta', 0.001) # model parameters
config.add_model_parameter('gamma', 0.2)
config.add_model_parameter("fraction_infected", 0.05)

num_nodes = 2500
a = 4 # parameter for skewed distribution values

for num_graphs in range(1): # independent graphs to test
    graph_num.append(num_graphs)
    G = nx.watts_strogatz_graph(2500, 250, 0.1) # generating the graph
    
    keys = np.arange(num_nodes) # drawing from the distribution and binning
    values = skewnorm.rvs(a, size=num_nodes)
    num_bins = 50 
    
    digitized = np.digitize(values, np.linspace(min(values), max(values), num=num_bins)) # labelling nodes by their bins

    node_mapping = dict(zip(G.nodes(), sorted(G.nodes(), key=lambda k: random.random()))) # randomly relabelling nodes
    G = nx.relabel_nodes(G, node_mapping)
    
    model = ep.SIRModel(G) # configuring model
    model.set_initial_status(config)

    trends = multi_runs(model, execution_number=10, iteration_number=2500, infection_sets=None, nprocesses=5) # running model on initial graph

    # calculating results of model on unaggregated graph
    full_res.append([(len(list(G.nodes())) - x)/len(list(G.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]])
    stability_full.append([trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)])
    peak_inf_full.append([max(x)/len(list(G.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]])
    peak_time_full.append([trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)])
    
    
    
    g_cop = G.copy() # copy of graph to aggregate
    join_perc = 0.8 # shared neighbour threshold parameter
    for i in range(1,num_bins+1): # doing pairwise comparisons bin by bin
        relevant_nodes = keys[digitized==i]
        for j in relevant_nodes:
            if (j in g_cop.nodes()): 
                for k in relevant_nodes:
                    if (k in g_cop.nodes()) and (k != j): # if nodes haven't been removed by aggregation
                        if shared_neighbours(g_cop, j, k, min_share=False) > join_perc: # use larger proportion, join if they exceed
                            g_cop = join_nodes(g_cop, j, k)

    model = ep.SIRModel(g_cop)
    model.set_initial_status(config)

    trends = multi_runs(model, execution_number=10, iteration_number=2500, infection_sets=None, nprocesses=5) # running model on shared neighbour aggregated graph

    non_rand_agg.append([(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]])
    stability_non_rand.append([trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)])
    peak_inf_non_rand.append([max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]])
    peak_time_non_rand.append([trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)])
    
    
    non_rand_ave_deg.append(sum(nx.degree_centrality(g_cop).values())/len(g_cop.nodes()))
    
    combinations = num_nodes - len(g_cop.nodes()) # number of aggregations that took place

    nodes_aggregated.append(combinations)

    g_cop = G.copy() # another copy of graph to do random aggregation
    for j in range(combinations): # joining nodes at random
        all_nodes = list(g_cop.nodes())
        choices = np.random.choice(range(len(all_nodes)), 2, replace=False)
        g_cop = join_nodes(g_cop, all_nodes[choices[0]], all_nodes[choices[1]])

    model = ep.SIRModel(g_cop)
    model.set_initial_status(config)

    trends = multi_runs(model, execution_number=10, iteration_number=2500, infection_sets=None, nprocesses=5) # running model on randomly aggregated graph

    rand_agg.append([(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]])
    stability_rand.append([trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)])
    peak_inf_rand.append([max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]])
    peak_time_rand.append([trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)])
    
    
    rand_ave_deg.append(sum(nx.degree_centrality(g_cop).values())/len(g_cop.nodes()))
    
aggregation_results = pd.DataFrame({'graph_num':graph_num, 'nodes_aggregated': nodes_aggregated, 'full_res': full_res, 'non_rand_agg': non_rand_agg, 'rand_agg': rand_agg, 'non_rand_ave_deg': non_rand_ave_deg, 'rand_ave_deg': rand_ave_deg,
                                   'peak_inf_full': peak_inf_full, 'peak_inf_non_rand': peak_inf_non_rand, 'peak_inf_rand': peak_inf_rand, 'stability_full': stability_full,
                                   'stability_non_rand': stability_non_rand, 'stability_rand': stability_rand, 'peak_time_full': peak_time_full, 'peak_time_non_rand': peak_time_non_rand, 'peak_time_rand': peak_time_rand})

In [4]:
aggregation_results

,graph_num,nodes_aggregated,full_res,non_rand_agg,rand_agg,non_rand_ave_deg,rand_ave_deg,peak_inf_full,peak_inf_non_rand,peak_inf_rand,stability_full,stability_non_rand,stability_rand,peak_time_full,peak_time_non_rand,peak_time_rand
0,0,841,"[0.414, 0.4484, 0.416, 0.5028, 0.4816, 0.5508,...","[0.21760096443640747, 0.35443037974683544, 0.3...","[0.6305003013863774, 0.6582278481012658, 0.719...",0.117581,0.200351,"[0.0708, 0.0704, 0.0628, 0.0672, 0.07, 0.082, ...","[0.050030138637733576, 0.060880048221820374, 0...","[0.13743218806509946, 0.14647377938517178, 0.1...","[72, 86, 67, 90, 87, 73, 74, 87, 92, 98]","[51, 64, 84, 73, 69, 62, 60, 68, 53, 90]","[50, 41, 58, 68, 58, 47, 49, 59, 49, 61]","[10, 15, 15, 17, 12, 20, 5, 24, 10, 14]","[1, 8, 2, 10, 0, 6, 0, 6, 0, 0]","[12, 14, 14, 14, 15, 16, 15, 16, 15, 11]"


Random Edge Removal, Threshold Model

In [5]:
all_results_df = pd.DataFrame(columns=['average_degree', 'num_infected', 'ave_num_infected', 'graph_num'])

for graph_num in range(1): # independent graphs to test

    g_cop = nx.gnm_random_graph(2500, 312500) # Creating the graph

    total_edges = len(list(g_cop.edges()))
    edges_to_remove = total_edges-1250
    remove_per_it = np.floor(edges_to_remove/50) # Number of edges to remove in 50 batches to have 1250 edges remaining (ave deg 1)

    config = mc.Configuration() # Setting model parameters
    frac_inf = 0.36
    config.add_model_parameter('fraction_infected', frac_inf)
    for i in g_cop.nodes():
            config.add_node_configuration("threshold", i, 0.50)

    average_degree = [sum(nx.degree_centrality(g_cop).values())/2500]

    model = ep.ThresholdModel(g_cop)
    model.set_initial_status(config)

    trends = multi_runs(model, execution_number=10, iteration_number=30, infection_sets=None, nprocesses=5) # running the model

    num_infected = [[trends[j]['trends']['node_count'][1][-1] for j in range(10)]]

    for i in range(50): # removing the edges in batches and running the model at each step
        all_edges = list(g_cop.edges())
        choices = np.random.choice(range(len(all_edges)), int(remove_per_it), replace=False)
        for choice in choices: # randomly removing edges
            g_cop.remove_edge(all_edges[choice][0], all_edges[choice][1])
        average_degree.append(sum(nx.degree_centrality(g_cop).values())/2500)

        model = ep.ThresholdModel(g_cop)
        model.set_initial_status(config)

        trends = multi_runs(model, execution_number=10, iteration_number=30, infection_sets=None, nprocesses=5) # running the model on the graph with edges removed

        num_infected.append([trends[j]['trends']['node_count'][1][-1] for j in range(10)])

    df = pd.DataFrame({'average_degree': average_degree, 'num_infected':num_infected})
    df['ave_num_infected'] = df['num_infected'].apply(lambda col: np.array(col).mean())
    df['graph_num'] = graph_num
    
    all_results_df = pd.concat([all_results_df, df])

In [ ]:
all_results_df

Random Node Removal, SIR Model

In [7]:
all_results_df = pd.DataFrame(columns=['num_infected', 'ave_num_infected', 'stability_time', 'ave_stability_time','peak_infected','peak_infected_time','ave_peak_infected','ave_peak_infected_time','graph_num'])

for set_num in range(1): # independent graphs to test
    for graph_num in range(1):
        g_cop = nx.barabasi_albert_graph(2500,132) # creating graphs

        all_nodes = list(g_cop.nodes())

        choices = np.random.choice(range(len(all_nodes)),1250, replace=False) # randomly removing half of the nodes first since the model always fully infects above this point
        g_cop.remove_nodes_from(choices)

        total_nodes = len(list(g_cop.nodes()))

        remove_per_it = np.floor((total_nodes-0.05*2500)/35) # number of nodes to remove in 35 batches to have 5% remaining at the end

        config = mc.Configuration() # model parameters
        config.add_model_parameter('gamma', 0.01)
        config.add_model_parameter("fraction_infected", 0.05)
        config.add_model_parameter('beta', 0.001)

        model = ep.SIRModel(g_cop)
        model.set_initial_status(config)

        trends = multi_runs(model, execution_number=10, iteration_number=1500, infection_sets=None, nprocesses=5) # running the model

        num_infected = [[(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]]]
        stability_time = [[trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)]]

        peak_infected =[[max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]]]
        peak_infected_time = [[trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)]]

        nodes = [len(list(g_cop.nodes()))]
        for i in range(35): # removing nodes in 35 batches
            all_nodes = list(g_cop.nodes())
            choices = np.random.choice(all_nodes, int(remove_per_it), replace=False)
            g_cop.remove_nodes_from(choices) # removing nodes at random

            model = ep.SIRModel(g_cop)
            model.set_initial_status(config)

            trends = multi_runs(model, execution_number=10, iteration_number=1500, infection_sets=None, nprocesses=5) # running the model after removing the nodes

            num_infected.append([(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]])
            stability_time.append([trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)])

            peak_infected.append([max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]])
            peak_infected_time.append([trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)])

            nodes.append(len(list(g_cop.nodes())))
        df = pd.DataFrame({'nodes': nodes, 'num_infected':num_infected, 'stability_time':stability_time, 'peak_infected': peak_infected, 'peak_infected_time': peak_infected_time})
        df['ave_num_infected'] = df['num_infected'].apply(lambda col: np.array(col).mean())
        df['ave_stability_time'] = df['stability_time'].apply(lambda col: np.array(col).mean())
        df['ave_peak_infected'] = df['peak_infected'].apply(lambda col: np.array(col).mean())
        df['ave_peak_infected_time'] = df['peak_infected_time'].apply(lambda col: np.array(col).mean())

        df['graph_num'] = graph_num + set_num*4

        all_results_df = pd.concat([all_results_df, df])

In [ ]:
all_results_df

Random Node Removal, Arxiv dataset

In [2]:
arxiv_G = nx.read_edgelist("cit-HepTh.txt.gz")
arxiv_G = arxiv_G.to_undirected()
arxiv_G = nx.convert_node_labels_to_integers(arxiv_G)

In [6]:
2*len(arxiv_G.edges())/len(list(arxiv_G.nodes()))/len(list(arxiv_G.nodes()))

0.000913735428202936

In [9]:
# http://snap.stanford.edu/data/cit-HepTh.html

arxiv_G = nx.read_edgelist("cit-HepTh.txt.gz")
arxiv_G = arxiv_G.to_undirected()
arxiv_G = nx.convert_node_labels_to_integers(arxiv_G)

all_results_df = pd.DataFrame(columns=['num_infected', 'ave_num_infected', 'stability_time', 'ave_stability_time','peak_infected','peak_infected_time','ave_peak_infected','ave_peak_infected_time','sim_num'])
total_nodes = len(list(arxiv_G.nodes()))

for sim_num in range(1): # independent simulations of graph degradation
    g_cop = arxiv_G.copy() # creating copy of graph

    remove_per_it = np.floor((total_nodes-0.05*total_nodes)/35) # number of nodes to remove in 35 batches to have 5% remaining at the end

    config = mc.Configuration() # model parameters
    config.add_model_parameter('gamma', 0.01)
    config.add_model_parameter("fraction_infected", 0.05)
    config.add_model_parameter('beta', 0.015)

    model = ep.SIRModel(g_cop)
    model.set_initial_status(config)

    trends = multi_runs(model, execution_number=10, iteration_number=1750, infection_sets=None, nprocesses=5) # running the model

    num_infected = [[(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]]]
    stability_time = [[trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)]]

    peak_infected =[[max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]]]
    peak_infected_time = [[trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)]]

    nodes = [len(list(g_cop.nodes()))]
    for i in range(35): # removing nodes in 35 batches
        all_nodes = list(g_cop.nodes())
        choices = np.random.choice(all_nodes, int(remove_per_it), replace=False)
        g_cop.remove_nodes_from(choices) # removing nodes at random

        model = ep.SIRModel(g_cop)
        model.set_initial_status(config)

        trends = multi_runs(model, execution_number=10, iteration_number=1750, infection_sets=None, nprocesses=5) # running the model after removing the nodes

        num_infected.append([(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]])
        stability_time.append([trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)])

        peak_infected.append([max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]])
        peak_infected_time.append([trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)])

        nodes.append(len(list(g_cop.nodes())))
    df = pd.DataFrame({'nodes': nodes, 'num_infected':num_infected, 'stability_time':stability_time, 'peak_infected': peak_infected, 'peak_infected_time': peak_infected_time})
    df['ave_num_infected'] = df['num_infected'].apply(lambda col: np.array(col).mean())
    df['ave_stability_time'] = df['stability_time'].apply(lambda col: np.array(col).mean())
    df['ave_peak_infected'] = df['peak_infected'].apply(lambda col: np.array(col).mean())
    df['ave_peak_infected_time'] = df['peak_infected_time'].apply(lambda col: np.array(col).mean())

    df['sim_num'] = sim_num

    all_results_df = pd.concat([all_results_df, df])

In [ ]:
all_results_df

SlashDot.com network processing, as in Wang et al. (2012)

In [7]:
# http://snap.stanford.edu/data/soc-sign-Slashdot081106.html
# Need to unzip the file to a txt and delete meta information first

txt_file = pd.read_csv('soc-sign-Slashdot081106.txt', delimiter = '\t')
txt_file = txt_file[['0','1']].loc[txt_file['1.1']!=-1] # removing all negative edges 
txt_file.to_csv('soc_updated.txt', index=False, sep = '\t')

soc_G = nx.read_edgelist("soc_updated.txt")
soc_G = soc_G.to_undirected()
soc_G = nx.convert_node_labels_to_integers(soc_G)

In [8]:
2*len(soc_G.edges())/len(list(soc_G.nodes()))/len(list(soc_G.nodes()))

0.00014279708222539718

Random Node Removal, SlashDot.com network

In [13]:
all_results_df = pd.DataFrame(columns=['num_infected', 'ave_num_infected', 'stability_time', 'ave_stability_time','peak_infected','peak_infected_time','ave_peak_infected','ave_peak_infected_time','sim_num'])
total_nodes = len(list(soc_G.nodes()))

for sim_num in range(1): # independent simulations of graph degradation
    g_cop = soc_G.copy() # creating copy of graph

    remove_per_it = np.floor((total_nodes-0.05*total_nodes)/35) # number of nodes to remove in 35 batches to have 5% remaining at the end

    config = mc.Configuration() # model parameters
    config.add_model_parameter('gamma', 0.01)
    config.add_model_parameter("fraction_infected", 0.05)
    config.add_model_parameter('beta', 0.085)

    model = ep.SIRModel(g_cop)
    model.set_initial_status(config)

    trends = multi_runs(model, execution_number=10, iteration_number=1750, infection_sets=None, nprocesses=5) # running the model

    num_infected = [[(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]]]
    stability_time = [[trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)]]

    peak_infected =[[max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]]]
    peak_infected_time = [[trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)]]

    nodes = [len(list(g_cop.nodes()))]
    for i in range(35): # removing nodes in 35 batches
        all_nodes = list(g_cop.nodes())
        choices = np.random.choice(all_nodes, int(remove_per_it), replace=False)
        g_cop.remove_nodes_from(choices) # removing nodes at random

        model = ep.SIRModel(g_cop)
        model.set_initial_status(config)

        trends = multi_runs(model, execution_number=10, iteration_number=1750, infection_sets=None, nprocesses=5) # running the model after removing the nodes

        num_infected.append([(len(list(g_cop.nodes())) - x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][0][-1] for j in range(10)]])
        stability_time.append([trends[j]['trends']['node_count'][0].index(trends[j]['trends']['node_count'][0][-1]) for j in range(10)])

        peak_infected.append([max(x)/len(list(g_cop.nodes())) for x in [trends[j]['trends']['node_count'][1] for j in range(10)]])
        peak_infected_time.append([trends[j]['trends']['node_count'][1].index(max(trends[j]['trends']['node_count'][1])) for j in range(10)])

        nodes.append(len(list(g_cop.nodes())))
    df = pd.DataFrame({'nodes': nodes, 'num_infected':num_infected, 'stability_time':stability_time, 'peak_infected': peak_infected, 'peak_infected_time': peak_infected_time})
    df['ave_num_infected'] = df['num_infected'].apply(lambda col: np.array(col).mean())
    df['ave_stability_time'] = df['stability_time'].apply(lambda col: np.array(col).mean())
    df['ave_peak_infected'] = df['peak_infected'].apply(lambda col: np.array(col).mean())
    df['ave_peak_infected_time'] = df['peak_infected_time'].apply(lambda col: np.array(col).mean())

    df['sim_num'] = sim_num

    all_results_df = pd.concat([all_results_df, df])

In [ ]:
all_results_df